In [1]:
#!pip install git+https://github.com/nockchun/rspy --force
import rspy as rsp
rsp.setSystemWarning(off=True)

In [2]:
import os
from glob import glob
import pandas as pd

# 시가총액(marcap) 가져오기

In [3]:
DATA_YEAR = "2010"

In [4]:
df = pd.read_csv(f'/home/jovyan/marcap/data/marcap-{DATA_YEAR}.csv.gz', dtype={'Code':str}, parse_dates=['Date'])

In [5]:
df.head()

,Code,Name,Close,Changes,ChagesRatio,Volume,Amount,Open,High,Low,Marcap,MarcapRatio,Stocks,ForeignShares,ForeignRatio,Rank,Date
0,005930,삼성전자,958000,9000,1.0,265562,254246714088,955000,966000,950000,141112764846000,11.27,147299337,74518914.0,50.59,1,2011-01-03
1,005490,POSCO,486000,-1000,-0.2,170542,82998761000,487000,489500,482000,42372801810000,3.38,87186835,43377735.0,49.75,2,2011-01-03
2,005380,현대차,177000,3500,2.0,556135,97468816500,174000,177000,173500,38988936783000,3.11,220276479,94245489.0,42.79,3,2011-01-03
3,009540,현대중공업,435000,-8000,-1.8,256635,111611214500,443000,443500,431000,33060000000000,2.64,76000000,15350207.0,20.20,4,2011-01-03
4,012330,현대모비스,294500,10000,3.5,212902,61730122000,287500,294500,285000,28667767653500,2.29,97343863,46134411.0,47.39,5,2011-01-03


In [18]:
df.columns

Index(['Code', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Amount',
       'Changes', 'ChagesRatio', 'Stocks', 'Marcap', 'MarcapRatio',
       'ForeignShares', 'ForeignRatio', 'Rank'],
      dtype='object')

In [17]:
"Name" in df.columns

False

# 데이터 전처리

In [6]:
df.fillna(0, inplace=True)
df = df[[
    "Code", "Date", "Open", "High", "Low", "Close", "Volume", "Amount", "Changes", "ChagesRatio", "Stocks", "Marcap", "MarcapRatio", "ForeignShares", "ForeignRatio", "Rank"
]]
df = df.astype({
    "Date":"str", "ForeignShares":"int64"
})

* Code : 종목코드
* Date : 날짜 (DatetimeIndex)
* Open : 시가
* High : 고가
* Low : 저가
* Close : 종가
* Volume : 거래량
* Amount : 거래대금
* Changes : 전일대비
* ChagesRatio : 전일비
* Marcap : 시가총액(백만원)
* Stocks : 상장주식수
* MarcapRatio : 시가총액비중(%)
* ForeignShares : 외국인 보유주식수
* ForeignRatio : 외국인 지분율(%)
* Rank: 시가총액 순위 (당일)

In [7]:
df.head()

,Code,Date,Open,High,Low,Close,Volume,Amount,Changes,ChagesRatio,Stocks,Marcap,MarcapRatio,ForeignShares,ForeignRatio,Rank
0,005930,2011-01-03,955000,966000,950000,958000,265562,254246714088,9000,1.0,147299337,141112764846000,11.27,74518914,50.59,1
1,005490,2011-01-03,487000,489500,482000,486000,170542,82998761000,-1000,-0.2,87186835,42372801810000,3.38,43377735,49.75,2
2,005380,2011-01-03,174000,177000,173500,177000,556135,97468816500,3500,2.0,220276479,38988936783000,3.11,94245489,42.79,3
3,009540,2011-01-03,443000,443500,431000,435000,256635,111611214500,-8000,-1.8,76000000,33060000000000,2.64,15350207,20.20,4
4,012330,2011-01-03,287500,294500,285000,294500,212902,61730122000,10000,3.5,97343863,28667767653500,2.29,46134411,47.39,5


# DB에 저장.

In [8]:
rows = []
for index, row in df.iterrows():
    rows.append(tuple(row))

In [9]:
rows[:1]

[('005930',
  '2011-01-03',
  955000,
  966000,
  950000,
  958000,
  265562,
  254246714088,
  9000,
  1.0,
  147299337,
  141112764846000,
  11.27,
  74518914,
  50.59,
  1)]

In [10]:
db = rsp.PGManager()

In [11]:
db.createPool("postgres", "postgres", "postgres", "192.168.0.200")
db.setMapper("mapper.xml")

In [12]:
db.executeMany("inMarcap", rows)